In [ ]:
%matplotlib widget

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
from ..utils.math.vectors import Vector2D
from matplotlib.patches import Circle
import random


class FeaturePlotEventHandler:
    def __init__(self, event_modifier_key):
        self.event_modifier_key = event_modifier_key
        # there may be better alternatives to this class persisting the features
        self.features = []

    def __call__(self, event):
        if event.key != self.event_modifier_key:
            return
        self.features.append((event.xdata, event.ydata))
        render()

# globals
GLOBAL_VECTOR_OFFSET = (0.4, 0.4)
AX_VIEWPORT = (-0.5, 1)
gvo_vec = Vector2D(*GLOBAL_VECTOR_OFFSET)
# figure, axes, and event handlers
fig, ax = plt.subplots()
ind = np.linspace(*AX_VIEWPORT)  # reusable linear space
plot1 = ax.scatter([], [], color="b", marker="o")
plot2 = ax.scatter([], [], color="r", marker="o")
fpeh1 = FeaturePlotEventHandler(None)
fpeh2 = FeaturePlotEventHandler("shift")
cid1 = fig.canvas.mpl_connect("button_press_event", fpeh1)
cid2 = fig.canvas.mpl_connect("button_press_event", fpeh2)
# w vector
# TODO: make sure to get rid of gvo_vec mention
quiver = ax.quiver(0, 0, gvo_vec.x, gvo_vec.y, angles="xy", scale_units="xy", scale=1)
# hyperplane
(hyperplane_plot,) = ax.plot(ind, ind, ":g")
# section filling
pos_fill = ax.fill_between(ind, 0, 0, facecolor="b", alpha=0.2)
neg_fill = ax.fill_between(ind, 0, 0, facecolor="r", alpha=0.2)
# test point indicator
circle = Circle((0, 0), 0.03, fill=False, color="g", label="test point")
# text
text = ax.text(AX_VIEWPORT[0] + 0.02, AX_VIEWPORT[0] + 0.02, "")
# w vector
w = Vector2D(0, 0)


def init_ax():
    # initialize the figure, plots, and feature plotters
    ax.set_xlim(*AX_VIEWPORT)
    ax.set_ylim(*AX_VIEWPORT)
    ax.set_aspect("equal")
    ax.autoscale(False)
    ax.set_title(
        "Perceptron Feature Plotter\n(click to add points, hold shift for red)"
    )
    ax.axhline(y=0)
    ax.axvline(x=0)
    ax.add_patch(circle)
    # create w vector

    global w
    w = Vector2D(11.8, 1.9)
    w.normalize()
    w.scale(0.5)


def render():
    # update w
    quiver.set_UVC(w.x, w.y)

    # update features
    if fpeh1.features:
        plot1.set_offsets(fpeh1.features)
    if fpeh2.features:
        plot2.set_offsets(fpeh2.features)

    # update hyperplane
    w_vector_slope_rot = -w.x / w.y  # swap w.x, w.y; w.x *= -1
    # dep = w_vector_slope_rot * (ind - gvo_vec.x) + gvo_vec.y
    dep = w_vector_slope_rot * ind
    hyperplane_plot.set_data(ind, dep)

    # update section fills
    global pos_fill
    global neg_fill
    pos_fill.remove()
    neg_fill.remove()
    pos_fill = ax.fill_between(
        ind, dep.tolist(), AX_VIEWPORT[int(w.y > 0)], facecolor="b", alpha=0.2
    )
    neg_fill = ax.fill_between(
        ind, dep.tolist(), AX_VIEWPORT[int(w.y < 0)], facecolor="r", alpha=0.2
    )

    # update text
    text.set_text(f"magnitude of w: {round(w.magnitude(), 5)}")

    # redraw canvas
    ax.figure.canvas.draw()


init_ax()
render()

# claim 1: since hyperplane offset (in y direction w.l.o.g.) depends on w_y,
# w_z can be literally anything because I can find a w_y to compensate


In [ ]:
import random
from ..utils.helpers import data_print
from colorama import Fore

LABEL_SPACE = (-1, 1)

# data will be of the form ((x, y), laassociationbel)
data = []
data.extend([(Vector2D(x, y), LABEL_SPACE[1]) for x, y in plot1.get_offsets()])
data.extend([(Vector2D(x, y), LABEL_SPACE[0]) for x, y in plot2.get_offsets()])
random.shuffle(data)


def perceptron():
    global w
    misses = 0
    for vec, label in data:
        miss = False
        circle.center = vec.x, vec.y # + gvo_vec.x, vec.y + gvo_vec.y
        ax.figure.canvas.draw()
        input(f"about to test feature: {vec}")
        if label * w.dot_product(vec) <= 0:
            misses += 1
            miss = True
        labeled_data = {
            "current w\u20D7": w,
            "miss count": misses,
            "testing feature": vec,
            "want": label,
            "got": w.dot_product(vec),
            "result": "miss! adjusting w\u20D7" if miss else "hit",
        }
        for line in data_print(labeled_data, Fore.BLUE):
            print(line)
        input(f"done testing: {'miss' if miss else 'hit'}")
        if miss:
            # !CAUTION! this is updating the data in memory!!!
            vec.scale(label)
            w.add(vec)
            render()

perceptron()


In [ ]:
# plot2.get_offsets()
# quiver.set_UVC(0.28688, -0.45504)
# list(zip(*fp.plot1_points))
# list(zip((1,3),(5,2),(345,23)))

list(zip((1,2),(3,4),(64,2345)))